# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 31 2022 11:48AM,249697,19,ACG-2102491724,ACG North America LLC,Released
1,Oct 31 2022 11:20AM,249693,16,8715335,Sartorius Bioprocess Solutions,Released
2,Oct 31 2022 11:20AM,249692,19,ADV80007174,"AdvaGen Pharma, Ltd",Released
3,Oct 31 2022 11:20AM,249692,19,ADV80007175,"AdvaGen Pharma, Ltd",Released
4,Oct 31 2022 11:20AM,249692,19,ADV80007176,"AdvaGen Pharma, Ltd",Released
5,Oct 31 2022 11:20AM,249692,19,ADV80007177,"AdvaGen Pharma, Ltd",Released
6,Oct 31 2022 11:20AM,249692,19,ADV80007178,"AdvaGen Pharma, Ltd",Released
7,Oct 31 2022 11:20AM,249692,19,ADV80007179,"AdvaGen Pharma, Ltd",Released
8,Oct 31 2022 11:20AM,249692,19,ADV80007180,"AdvaGen Pharma, Ltd",Released
9,Oct 31 2022 11:20AM,249692,19,ADV80007181,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,249689,Released,1
17,249691,Released,1
18,249692,Released,19
19,249693,Released,1
20,249697,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249689,NaN,NaN,1.0
249691,NaN,NaN,1.0
249692,NaN,NaN,19.0
249693,NaN,NaN,1.0
249697,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249651,2.0,15.0,1.0
249670,0.0,0.0,49.0
249671,0.0,36.0,27.0
249678,0.0,0.0,1.0
249679,0.0,7.0,24.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249651,2,15,1
249670,0,0,49
249671,0,36,27
249678,0,0,1
249679,0,7,24


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249651,2,15,1
1,249670,0,0,49
2,249671,0,36,27
3,249678,0,0,1
4,249679,0,7,24


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249651,2,15,1
1,249670,,,49
2,249671,,36,27
3,249678,,,1
4,249679,,7,24


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 31 2022 11:48AM,249697,19,ACG North America LLC
1,Oct 31 2022 11:20AM,249693,16,Sartorius Bioprocess Solutions
2,Oct 31 2022 11:20AM,249692,19,"AdvaGen Pharma, Ltd"
21,Oct 31 2022 11:16AM,249691,12,Hush Hush
22,Oct 31 2022 11:12AM,249689,10,"Methapharm, Inc."
23,Oct 31 2022 11:07AM,249688,10,"Citieffe, Inc."
26,Oct 31 2022 10:59AM,249687,10,"Senseonics, Incorporated"
27,Oct 31 2022 10:46AM,249686,19,"AdvaGen Pharma, Ltd"
28,Oct 31 2022 10:33AM,249683,19,"Innogenix, LLC"
51,Oct 31 2022 9:38AM,249681,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 31 2022 11:48AM,249697,19,ACG North America LLC,,,1
1,Oct 31 2022 11:20AM,249693,16,Sartorius Bioprocess Solutions,,,1
2,Oct 31 2022 11:20AM,249692,19,"AdvaGen Pharma, Ltd",,,19
3,Oct 31 2022 11:16AM,249691,12,Hush Hush,,,1
4,Oct 31 2022 11:12AM,249689,10,"Methapharm, Inc.",,,1
5,Oct 31 2022 11:07AM,249688,10,"Citieffe, Inc.",,3,
6,Oct 31 2022 10:59AM,249687,10,"Senseonics, Incorporated",,,1
7,Oct 31 2022 10:46AM,249686,19,"AdvaGen Pharma, Ltd",,,1
8,Oct 31 2022 10:33AM,249683,19,"Innogenix, LLC",,1,22
9,Oct 31 2022 9:38AM,249681,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 31 2022 11:48AM,249697,19,ACG North America LLC,1,,
1,Oct 31 2022 11:20AM,249693,16,Sartorius Bioprocess Solutions,1,,
2,Oct 31 2022 11:20AM,249692,19,"AdvaGen Pharma, Ltd",19,,
3,Oct 31 2022 11:16AM,249691,12,Hush Hush,1,,
4,Oct 31 2022 11:12AM,249689,10,"Methapharm, Inc.",1,,
5,Oct 31 2022 11:07AM,249688,10,"Citieffe, Inc.",,3,
6,Oct 31 2022 10:59AM,249687,10,"Senseonics, Incorporated",1,,
7,Oct 31 2022 10:46AM,249686,19,"AdvaGen Pharma, Ltd",1,,
8,Oct 31 2022 10:33AM,249683,19,"Innogenix, LLC",22,1,
9,Oct 31 2022 9:38AM,249681,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 31 2022 11:48AM,249697,19,ACG North America LLC,1,,
1,Oct 31 2022 11:20AM,249693,16,Sartorius Bioprocess Solutions,1,,
2,Oct 31 2022 11:20AM,249692,19,"AdvaGen Pharma, Ltd",19,,
3,Oct 31 2022 11:16AM,249691,12,Hush Hush,1,,
4,Oct 31 2022 11:12AM,249689,10,"Methapharm, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 31 2022 11:48AM,249697,19,ACG North America LLC,1.0,NaN,NaN
1,Oct 31 2022 11:20AM,249693,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
2,Oct 31 2022 11:20AM,249692,19,"AdvaGen Pharma, Ltd",19.0,NaN,NaN
3,Oct 31 2022 11:16AM,249691,12,Hush Hush,1.0,NaN,NaN
4,Oct 31 2022 11:12AM,249689,10,"Methapharm, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 31 2022 11:48AM,249697,19,ACG North America LLC,1.0,0.0,0.0
1,Oct 31 2022 11:20AM,249693,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
2,Oct 31 2022 11:20AM,249692,19,"AdvaGen Pharma, Ltd",19.0,0.0,0.0
3,Oct 31 2022 11:16AM,249691,12,Hush Hush,1.0,0.0,0.0
4,Oct 31 2022 11:12AM,249689,10,"Methapharm, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2247094,105.0,61.0,2.0
12,249691,1.0,0.0,0.0
16,249693,1.0,0.0,0.0
19,998758,43.0,1.0,0.0
20,249680,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2247094,105.0,61.0,2.0
1,12,249691,1.0,0.0,0.0
2,16,249693,1.0,0.0,0.0
3,19,998758,43.0,1.0,0.0
4,20,249680,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,105.0,61.0,2.0
1,12,1.0,0.0,0.0
2,16,1.0,0.0,0.0
3,19,43.0,1.0,0.0
4,20,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,105.0
1,12,Released,1.0
2,16,Released,1.0
3,19,Released,43.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20
Status,,,,,
Completed,2.0,0.0,0.0,0.0,0.0
Executing,61.0,0.0,0.0,1.0,0.0
Released,105.0,1.0,1.0,43.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20
0,Completed,2.0,0.0,0.0,0.0,0.0
1,Executing,61.0,0.0,0.0,1.0,0.0
2,Released,105.0,1.0,1.0,43.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20
0,Completed,2.0,0.0,0.0,0.0,0.0
1,Executing,61.0,0.0,0.0,1.0,0.0
2,Released,105.0,1.0,1.0,43.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()